In [1]:
!pip install xlsxwriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.4 MB/s eta 0:00:00


In [1]:

import pandas as pd

In [33]:
summary_manual_annotations_automaxo_path = '../results/evaluation_summary_post_ontoGPT.tsv'
raw_text_manual_annotated_articles_path = '../evaluation_no_replacement.tsv'
detailed_manual_annotations_automaxo_path = '../results/evaluation_detailed_post_ontoGPT.json'


In [34]:
summary_manual_annotations_automaxo_df = pd.read_csv(summary_manual_annotations_automaxo_path, delimiter='\t')
raw_text_manual_annotated_articles_df = pd.read_csv(raw_text_manual_annotated_articles_path, delimiter='\t',  header=None)
raw_text_manual_annotated_articles_df.columns = ['Citation', 'Relationship', 'Text']
raw_text_manual_annotated_articles_df = raw_text_manual_annotated_articles_df.drop('Relationship', axis=1)

In [35]:
# save raw text and abstracts
raw_text_manual_annotated_articles_df.to_excel('raw_text_manual_annotated_articles.xlsx', engine='openpyxl', index=False)

In [36]:
# Merge the dataframes on 'Citation'
# merged_df = pd.merge(summary_manual_annotations_automaxo_df, raw_text_manual_annotated_articles_df, on='Citation', how='left')



In [37]:
summary_manual_annotations_automaxo_df.head()

,Citation,Subject,Subject Label,Non grounded Subject,Potential MAXO,Predicate,Object,Object Label,Non grounded Object,Potential HP,Qualifier,Qualifier Label,Non grounded Qualifier,Potential MONDO,Subject Qualifier,Subject Extension,Object Extension,Count
0,30691927,NaN,NaN,restitution therapies,[],treats,NaN,NaN,somatic growth failure,"[('HP:0001510', 'Growth failure')]",NaN,NaN,recessive gm3 synthase deficiency,"[('MONDO:0016019', 'GM3 synthase deficiency'),...",NaN,chebi:28892,NaN,1
1,30691927,maxo:0001298,therapies,NaN,[],treats,hp:0000253,progressive microcephaly,NaN,[],NaN,NaN,recessive gm3 synthase deficiency,"[('MONDO:0016019', 'GM3 synthase deficiency'),...",NaN,ganglioside restitution,NaN,1
2,30691927,NaN,NaN,ganglioside restitution therapies,[],treats,hp:0001263,global developmental delay,NaN,[],NaN,NaN,recessive gm3 synthase deficiency,"[('MONDO:0016019', 'GM3 synthase deficiency'),...",NaN,ganglioside restitution therapies,NaN,1
3,30691927,NaN,NaN,restitution therapies,[],treats,NaN,NaN,visual inattentiveness,[],NaN,NaN,recessive gm3 synthase deficiency,"[('MONDO:0016019', 'GM3 synthase deficiency'),...",NaN,chebi:28892,NaN,1
4,30691927,NaN,NaN,ganglioside restitution therapies,[],treats,NaN,NaN,dyskinetic movements,[],NaN,NaN,recessive gm3 synthase deficiency,"[('MONDO:0016019', 'GM3 synthase deficiency'),...",NaN,ganglioside,NaN,1


In [38]:
def expand_potentials(df):
    # Define the columns to be expanded
    columns_to_expand = ['Potential MAXO', 'Potential HP', 'Potential MONDO']

    # Process each specified column
    for column_name in columns_to_expand:
        # Initialize the new columns for expanded data
        df[f'{column_name}1'] = None
        df[f'{column_name}2'] = None

        # Iterate over each row in the DataFrame
        for index, row in df.iterrows():
            # Retrieve the current column's data
            items = eval(row[column_name]) if row[column_name] else None

            # Continue only if there is data in the column
            if items:
                # Removing exact duplicates
                seen = set()
                unique_items = []
                for item in items:
                    if item not in seen:
                        seen.add(item)
                        unique_items.append(item)

                # Assign the first unique item to the first new column if available
                if len(unique_items) > 0:
                    df.at[index, f'{column_name}1'] = unique_items[0][0], unique_items[0][1]
                if len(unique_items) > 1:
                    df.at[index, f'{column_name}2'] = unique_items[1][0], unique_items[1][1]

        # Check and drop the original column if all values are None (or the list was empty for all rows)
        if df[column_name].isnull().all():
            df.drop([column_name, f'{column_name}1', f'{column_name}2'], axis=1, inplace=True)
        else:
            df.drop(column_name, axis=1, inplace=True)

    return df


In [39]:
df_temp = summary_manual_annotations_automaxo_df.copy()

In [40]:
transformed_df = expand_potentials(df_temp)
transformed_df

,Citation,Subject,Subject Label,Non grounded Subject,Predicate,Object,Object Label,Non grounded Object,Qualifier,Qualifier Label,...,Subject Qualifier,Subject Extension,Object Extension,Count,Potential MAXO1,Potential MAXO2,Potential HP1,Potential HP2,Potential MONDO1,Potential MONDO2
0,30691927,NaN,NaN,restitution therapies,treats,NaN,NaN,somatic growth failure,NaN,NaN,...,NaN,chebi:28892,NaN,1,None,None,"(HP:0001510, Growth failure)",None,"(MONDO:0016019, GM3 synthase deficiency)","(MONDO:0018274, GM3 synthase deficiency)"
1,30691927,maxo:0001298,therapies,NaN,treats,hp:0000253,progressive microcephaly,NaN,NaN,NaN,...,NaN,ganglioside restitution,NaN,1,None,None,None,None,"(MONDO:0016019, GM3 synthase deficiency)","(MONDO:0018274, GM3 synthase deficiency)"
2,30691927,NaN,NaN,ganglioside restitution therapies,treats,hp:0001263,global developmental delay,NaN,NaN,NaN,...,NaN,ganglioside restitution therapies,NaN,1,None,None,None,None,"(MONDO:0016019, GM3 synthase deficiency)","(MONDO:0018274, GM3 synthase deficiency)"
3,30691927,NaN,NaN,restitution therapies,treats,NaN,NaN,visual inattentiveness,NaN,NaN,...,NaN,chebi:28892,NaN,1,None,None,None,None,"(MONDO:0016019, GM3 synthase deficiency)","(MONDO:0018274, GM3 synthase deficiency)"
4,30691927,NaN,NaN,ganglioside restitution therapies,treats,NaN,NaN,dyskinetic movements,NaN,NaN,...,NaN,ganglioside,NaN,1,None,None,None,None,"(MONDO:0016019, GM3 synthase deficiency)","(MONDO:0018274, GM3 synthase deficiency)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,32754920,NaN,NaN,calculated diet,prevents,NaN,NaN,lower body weight,NaN,NaN,...,NaN,calculated diet,NaN,1,"(MAXO:0000088, diet)",None,"(HP:0500001, BO)",None,"(MONDO:0002012, methylmalonic aciduria)","(MONDO:0011727, AN)"
230,32754920,NaN,NaN,amino acid supplements,prevents,NaN,NaN,biochemical disturbances,NaN,NaN,...,NaN,amino acid supplements,NaN,1,None,None,None,None,"(MONDO:0002012, methylmalonic aciduria)","(MONDO:0011727, AN)"
231,32389449,NaN,NaN,diet therapy,treats,NaN,NaN,intractable epilepsy,mondo:0010998,alg3-cdg,...,NaN,ketogenic,NaN,1,"(MAXO:0000088, diet)","(MAXO:0001298, therapy)","(HP:0001250, Epilepsy)",None,None,None
232,5658866,NaN,NaN,folic acid therapy,treats,NaN,NaN,subnormal serum folate levels,mondo:0004737,homocystinuria,...,with folic acid,chebi:27470,NaN,1,"(MAXO:0001298, therapy)",None,None,None,None,None


In [41]:
transformed_df.to_excel('transformed_df.xlsx', engine='openpyxl', index=False)
